# Deploy
* start all docker containers

In [ ]:
import NBInclude: @nbinclude
import JSON
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
function get_params()
    params = Dict{String,Dict{String,Any}}(
        "fetch_media_lists" => Dict("cpu" => 1, "memory" => 1, "concurrency" => 80),
        "compress_media_lists" => Dict("cpu" => 1, "memory" => 1, "concurrency" => 80),
        "bagofwords_jl" => Dict("cpu" => 2, "memory" => 4, "concurrency" => 80),
        "nondirectional" => Dict("cpu" => 2, "memory" => 4, "concurrency" => 80),
        "transformer_jl" => Dict("cpu" => 2, "memory" => 4, "concurrency" => 80),
        "ensemble" => Dict("cpu" => 2, "memory" => 4, "concurrency" => 80),
        "index" => Dict("cpu" => 2, "memory" => 2, "concurrency" => 80),
    )
    for medium in ALL_MEDIUMS
        for metric in ALL_METRICS
            params["bagofwords_py_$(medium)_$(metric)"] =
                Dict("cpu" => 8, "memory" => 8, "concurrency" => 1)
        end
    end
    for medium in ALL_MEDIUMS
        params["transformer_py_$(medium)"] =
            Dict("cpu" => 8, "memory" => 8, "concurrency" => 1)
    end
    for x in keys(params)
        params[x]["max_instances"] = Int(floor(100 / params[x]["cpu"]))
    end
    apps = collect(keys(params))
    apps = [["index"]; [x for x in apps if x != "index"]]
    for (i, x) in enumerate(apps)
        params[x]["port"] = 3000 + i - 1
    end
    params
end;

In [ ]:
function deploy(settings, params)
    if get(settings, "cloud_hosting", false)
        repo = settings["cloud_hosting_repo"]
        tag = settings["cloud_hosting_tag"]
        region = settings["cloud_hosting_region"]
        provider = settings["cloud_hosting_provider"]
        service_account = settings["cloud_hosting_service_account"]
        project = settings["cloud_hosting_project"]
        Threads.@threads for app in collect(keys(params))
            name = "$repo/$app:$tag"
            run(`docker tag rsys/$app $name`)
            run(`docker push $name`)
            cpus = params[app]["cpu"]
            memory = params[app]["memory"]
            max_instances = params[app]["max_instances"]
            cmdlist = [
                provider,
                "run",
                "deploy",
                replace(app, "_" => "-") * "-" * tag,
                "--image=$name",
                "--allow-unauthenticated",
                "--port=8080",
                "--service-account=$service_account",
                "--execution-environment=gen2",
                "--region=$region",
                "--project=$project",
                "--cpu=$cpus",
                "--memory=$(memory)Gi",
                "--max-instances=$max_instances",
            ]
            run(`$cmdlist`)
        end
    else
        try
            # check if network already exists
            run(pipeline(`docker network ls`, `grep -w rsys`))
        catch
            run(`docker network create rsys`)
        end
        for app in keys(params)
            port = params[app]["port"]
            cpus = params[app]["cpu"]
            memory = params[app]["memory"]
            cmd = [
                "docker",
                "run",
                "--rm",
                "--name",
                app,
                "-p",
                "$port:8080",
                "--cpus",
                "$cpus",
                "-m",
                "$(memory)g",
                "--network",
                "rsys",
                "rsys/$app",
            ]
            run(`$cmd`, wait = false)
        end
    end
end;

In [ ]:
deploy(get_settings(), get_params());